In [1]:
import pandas as pd
import numpy as np
import requests
import csv
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
!pip install folium
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


### Creating the dataframe

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove Not Assigned from Borough

In [3]:
df.drop(df[df["Borough"] == "Not assigned"].index, axis = 0, inplace = True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Replace "/" with "," and Index Reset, fyi - source list is already updated with this, making this a redundant task.

In [4]:
yyz_grouped = df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ", ".join(x))
yyz_grouped['Neighbourhood'] = yyz_grouped['Neighbourhood'].str.replace('/', ',')
yyz_grouped.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Replacing Not Assigned with Borough Name in Neighbourhood

In [5]:

for index in yyz_grouped.index:
    count = 0
    if yyz_grouped["Neighbourhood"][index] == '':
        yyz_grouped["Neighbourhood"][index] = yyz_grouped["Borough"][index]
        print(f"Borough {df.Borough[index]} added to Neighbourhood")
        count += 1

print("Number of changes made:", count)

Number of changes made: 0


### Print number of rows

In [6]:
yyz_grouped.shape

(103, 3)

### Adding Coordinates

In [7]:
coordinates_df = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
coordinates_df.rename(columns = {'Postal Code': 'Postal Code'},inplace = True)
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
coordinates_merged = pd.merge(coordinates_df, yyz_grouped, on= 'Postal Code')
coordinates_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


### Explore and cluster the neighborhoods in Toronto

In [10]:
yyz_explore=coordinates_merged[coordinates_merged['Borough'].str.contains("Toronto")]
yyz_explore.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
37,M4E,43.676357,-79.293031,East Toronto,The Beaches
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
42,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
43,M4M,43.659526,-79.340923,East Toronto,Studio District
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


In [11]:
yyz_explore["Borough"].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

### Toronto Coordinates and Map

In [12]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [13]:
yyz_map = folium.Map(location = [latitude, longitude], zoom_start = 12)
for lat, lng, borough, neighborhood in zip(yyz_explore['Latitude'], yyz_explore['Longitude'], yyz_explore['Borough'], yyz_explore['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(yyz_map)

yyz_map

In [14]:
CLIENT_ID = 'K0OC5YBMDWVAEUHUTNCH5QUBCJGKX4FHSGAQNX5EZ1YBJOON'
CLIENT_SECRET = '4LF3ELZ1T13JEXSY41S0OSVZIQ0HTEV5VGTO2J3YMNK5A4GA' 
VERSION = '20201124'
LIMIT = 30
radius = 500
print('My foursquare credentails are:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My foursquare credentails are:
CLIENT_ID: K0OC5YBMDWVAEUHUTNCH5QUBCJGKX4FHSGAQNX5EZ1YBJOON
CLIENT_SECRET:4LF3ELZ1T13JEXSY41S0OSVZIQ0HTEV5VGTO2J3YMNK5A4GA


In [15]:
search_query = 'Indian'
radius = 500
print(search_query + ' 500m OK!')

Indian 500m OK!


### Defining the URL

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
yyz_venues = getNearbyVenues(names=yyz_explore['Neighbourhood'],
                                   latitudes=yyz_explore['Latitude'],
                                   longitudes=yyz_explore['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

### Examining the results

In [18]:
yyz_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [19]:
yyz_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Airport,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
Airport Service,2,2,2,2,2,2
...,...,...,...,...,...,...
Vegetarian / Vegan Restaurant,8,8,8,8,8,8
Video Game Store,1,1,1,1,1,1
Vietnamese Restaurant,4,4,4,4,4,4


In [20]:
yyz_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


### One-hot encoding venue categories

In [21]:
yyz_onehot = pd.get_dummies(yyz_venues[['Venue Category']], prefix="", prefix_sep="")

yyz_onehot.insert(loc = 0, column = 'Neighbourhood', value = yyz_venues['Neighborhood'])
yyz_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
yyz_onehot.shape

(858, 197)

### Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category

In [23]:
yyz_grouped = yyz_onehot.groupby('Neighbourhood').mean().reset_index()
yyz_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.117647,0.058824,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333


In [24]:
yyz_grouped.shape

(39, 197)

### Printing each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in yyz_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = yyz_grouped[yyz_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.07
2            Beer Bar  0.07
3  Seafood Restaurant  0.07
4      Farmers Market  0.07


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1       Nightclub  0.08
2     Coffee Shop  0.08
3  Breakfast Spot  0.08
4             Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Auto Workshop  0.06
2            Skate Park  0.06
3         Burrito Place  0.06
4            Comic Shop  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0   Airport Lounge  0.12
1  Airport Service  0.12
2          Airport  0.06
3    Historic Site  0.06
4      Coffee Shop  0.06


----Central Bay Street----
                       venue  freq
0      

### Sort the venues in descending order

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = yyz_grouped['Neighbourhood']

for ind in np.arange(yyz_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(yyz_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Park,Tailor Shop,Jazz Club,Bistro,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Nightclub,Breakfast Spot,Climbing Gym,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Playground
2,"Business reply mail Processing Centre, South C...",Garden,Fast Food Restaurant,Skate Park,Light Rail Station,Brewery,Restaurant,Burrito Place,Recording Studio,Butcher,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Harbor / Marina,Sculpture Garden,Boutique,Rental Car Location,Boat or Ferry,Bar,Historic Site
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Park,Comic Shop,Poke Place


### Making Clusters for Neighborhood

In [28]:
# set number of clusters
kclusters = 5

yyz_grouped_clustering = yyz_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yyz_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labelsss', kmeans.labels_)

yyz_merged = yyz_explore

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
yyz_merged = yyz_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
yyz_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Cluster Labelsss,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,3,Neighborhood,Trail,Pub,Health Food Store,Yoga Studio,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Pizza Place,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Pub
42,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West",0,Park,Sandwich Place,Pub,Steakhouse,Ice Cream Shop,Sushi Restaurant,Burrito Place,Pizza Place,Gym,Fast Food Restaurant
43,M4M,43.659526,-79.340923,East Toronto,Studio District,0,Coffee Shop,Bakery,Café,American Restaurant,Comfort Food Restaurant,Bookstore,Brewery,Cheese Shop,Pet Store,Park
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,2,Bus Line,Swim School,Park,College Cafeteria,Dance Studio,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner


### Sort by Neighbourhoods

In [30]:
neighbourhoods_venues_sorted.head()

,Cluster Labelsss,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Park,Tailor Shop,Jazz Club,Bistro,Museum
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Nightclub,Breakfast Spot,Climbing Gym,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Playground
2,0,"Business reply mail Processing Centre, South C...",Garden,Fast Food Restaurant,Skate Park,Light Rail Station,Brewery,Restaurant,Burrito Place,Recording Studio,Butcher,Farmers Market
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Harbor / Marina,Sculpture Garden,Boutique,Rental Car Location,Boat or Ferry,Bar,Historic Site
4,0,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Park,Comic Shop,Poke Place


### Map Clusters

In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(yyz_merged['Latitude'], yyz_merged['Longitude'], yyz_merged['Neighbourhood'], yyz_merged['Cluster Labelsss']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters